# Questão 02

## Legenda das bases:
* **ID_CONTA:** Identificador da conta;
* **DT_ACORDO:** Data do acordo;
* **NU_DIAS_ATRASO:** Número de dias após o vencimento que esse cliente está sem pagar;
* **VALOR_CRELIQ:** Valor da dívida do cliente quando ele ultrapassa os 65 dias de atraso;
* **DIVIDA_ATUAL:** Dívida atualizada com juros;
* **RESPOSTA:** Se for 1, significa que o cliente aderiu aquele acordo. Ou seja, efetuou o pagamento da entrada. Se for 0, caso contrário.
* **QTD_PARCELAMENTO_XM:** Quantidade de parcelamentos de fatura aderidos X meses antes da fatura que ele deixou de pagar.
* **LIMITE:** Limite total do cliente;
* **QTD_EXTRATOS:** Quantidade de Faturas emitidas;
* **QTD_FX0:** Quantidade de faturas emitidas em dia;
* **QTD_FX1:** Quantidade de faturas emitidas no qual o cliente não pagou a fatura do mês anterior;
* **QTD_FX2:** Quantidade de faturas emitidas no qual o cliente não pagou as duas ultimas faturas anteriores;
* **QTD_ACIONAMENTO:** Quantidade de vezes que ligamos para os clientes para cobrar a sua dívida;
* **QTD_CP:** Quantidade de vezes que a ligação foi atendida, mas não houve contato com o cliente;
* **QTD_CPC:** Quantidade de vezes que o próprio cliente atendeu a ligação de cobrança.
* **DEFINIÇÃO:** Acordo é uma proposta de negociação da dívida por parte da Fortbrasil para clientes que estão com mais de 180 dias de atraso.


## Item 2.1
2.1 – Qual o percentual de adesão mensal por faixa de atraso (Histórico)? 

## Importações

In [1]:
## Manipulação dos dados
import pandas as pd

## Visualização dos dados
import plotly.graph_objects as go

## Função auxiliar

In [2]:
## Função para formatar a data e pegar apenas mês e ano
def date_format(date_serie):
    return date_serie.apply(lambda x : '-'.join(x.split('-')[0:2]))

## Leitura das bases

In [3]:
df = pd.read_csv('Questão 2 - Base 4.txt', sep='\t')

## Resolução da questão

In [4]:
## verificando os dados de df5
df.head()

,ID_CONTA,DT_ACORDO,QTD_EXTRATOS,QTD_FX0_GERAL,QTD_FX1_GERAL,QTD_FX2_GERAL,QTD_FX0_3M,QTD_FX0_6M,QTD_FX1_3M,QTD_FX1_6M,QTD_FX2_3M,QTD_FX2_6M
0,1385626,2018-11-22 00:00:00.000,5,5,0,0,4,6,0,0,0,0
1,1405852,2018-11-22 00:00:00.000,3,2,1,0,3,3,1,1,0,0
2,1163186,2018-11-22 00:00:00.000,26,23,2,1,4,5,0,1,0,1
3,1194908,2018-11-22 00:00:00.000,17,16,1,0,4,7,0,0,0,0
4,1419680,2018-11-22 00:00:00.000,2,2,0,0,3,3,0,0,0,0


In [5]:
## formatando a data em mes e ano
df['DT_ACORDO'] = date_format(df['DT_ACORDO'])

In [6]:
## vou verificar o intervalo que estou lidando
print(f'Intervalo de meses: { set( [ ano.split("-")[0] for ano in df["DT_ACORDO"] ] ) }')
print(f'Intervalo de anos:  {set(  [ mes.split("-")[1] for mes in df["DT_ACORDO"] ] )} ')

Intervalo de meses: {'2018', '2019'}
Intervalo de anos:  {'03', '04', '06', '11'} 


In [7]:
## vou agrupar os dados por id e mes
df_grouped = df[ ['DT_ACORDO',
                          'QTD_EXTRATOS',
                          'QTD_FX0_GERAL',
                          'QTD_FX1_GERAL',
                          'QTD_FX2_GERAL'] ].groupby(['DT_ACORDO']).sum().reset_index()

In [8]:
## hora de calcular as porcentagens
df_grouped['PORCENTAGEM_FX0'] = (df_grouped['QTD_FX0_GERAL'] / df_grouped['QTD_EXTRATOS']) * 100
df_grouped['PORCENTAGEM_FX1'] = (df_grouped['QTD_FX1_GERAL'] / df_grouped['QTD_EXTRATOS']) * 100
df_grouped['PORCENTAGEM_FX2'] = (df_grouped['QTD_FX2_GERAL'] / df_grouped['QTD_EXTRATOS']) * 100

In [9]:
## checando o novo conjunto
df_grouped

,DT_ACORDO,QTD_EXTRATOS,QTD_FX0_GERAL,QTD_FX1_GERAL,QTD_FX2_GERAL,PORCENTAGEM_FX0,PORCENTAGEM_FX1,PORCENTAGEM_FX2
0,2018-11,395495,358373,33595,3522,90.613788,8.494418,0.890530
1,2019-03,594377,534608,54116,5651,89.944261,9.104659,0.950743
2,2019-04,17584,15727,1687,170,89.439263,9.593949,0.966788
3,2019-06,751542,691819,54372,5347,92.053272,7.234725,0.711471


## Reposta da questão

In [11]:
X = df_grouped['DT_ACORDO']

fig = go.Figure(data=[
    go.Bar(name='FX1', x=X, y=list(df_grouped['PORCENTAGEM_FX1'].values)),
    go.Bar(name='FX2', x=X, y=df_grouped['PORCENTAGEM_FX2'].values)
])

## customizando
fig.update_layout(
    title_text='Relação entre FX1 e FX2',
    yaxis=dict(
        title='Quantidade (%)',
        titlefont_size=16,
        tickfont_size=14),
    xaxis=dict(
        title='Período',
        titlefont_size=16,
        tickfont_size=14,
    )
)


fig.update_layout(barmode='stack')
fig.show()

## Item 2.2

Qual modelo você utilizaria para traçar uma estratégia objetivando o aumento da adesão dos acordos? (Descreva a técnica utilizada)

Eu usaria o algoritmo de Gradient Boost.
**Motivos:**
* **Modelo consistente:** o Gradient Boos é um modelo muito utilizado pelos cientistas de dados e, geralmente, oferece resultados satisfatórios e difíceis de serem batidos;
* **Bem flexível:** pode otimizar diferentes funções de perda e oferece várias opções de ajuste de hiperparâmetros que tornam o ajuste do modelo muito flexível.
* **Pouco pré processamento:** o Gradient Boost precisa de pouco ou nenhum pré processamento em seus dados para que seja executado;
* **Dilui valores faltantes:** ele consegue lidar muito bem com valores faltantes, não sendo necessário inseri-los. Isso se dá pelos diversos cálculos dos diversos modelos fracos que vão diluindo as anomalidas causadas pela falta de alguns dados.


## Item 2.3
Quais indicadores e ferramentas você utilizaria para avaliar a performance/aderência desse modelo? (Descreva os indicadores utilizados)

Os algoritmos de Gradient Boost têm uma certa tendência ao overfitting. Para esse algoritmo, justamente por esse motivo, é recomendável usar a técnica de validação cruzada para medir e avaliar a capacidade do algoritmo de generalizar seus dados.
**Outros motivos**
* **Precisão:** a técnica de validação é uma das mais, senão a mais, precisa da hora de se avaliar um modelo;
* **Variedade:** permite o modelo ser treinado e avaliado por uma gama maior de conjuntos de dados se comparado com as formas normais de divisão, treino e teste dos dados.

**Funcionamento**
Na técnica de Validação Cruzada existem alguns métodos que dão certa variação ao método, como por exemplo, o método de validação cruzada k-folder, que seria meu método escolhido para esse problema.
O método de validação cruzada denominado k-fold consiste em dividir o conjunto total de dados em k subconjuntos de mesmo tamanho e, a partir daí, um subconjunto é utilizado para teste e os k-1 restantes são utilizados para estimação dos parâmetros, fazendo-se o cálculo da acurácia do modelo. Este processo é realizado k vezes alternando o subconjunto de teste.
 
 
Ao final das k iterações calcula-se a precisão sobre os erros encontrados, através da equação, obtendo assim uma medida mais confiável sobre a capacidade do modelo de generalizar seu aprendizado.


## Item 2.5
Crie um relatório analítico no Power BI para acompanhar a adesão dos acordos e criar insights de como melhorar essa adesão.

O relatório feito usando Power BI está disponível neste mesmo diretório, aqui irei expor meus insights sobre o relatório.

A partir das análises, conclui-se que a FortBrasil está com um está com um baixo índice de adesão dos contratos, apenas cerca de 3.87% dos clientes aderiram às propostas da empresa no período analisada.
 
Consegui observar uma pequena peculiaridade nos dados: o valor da dívida não influi tanto quanto se imagina. A diferença entre as médias das dívidas das pessoas que aderiram e as pessoas que não aderiram às propostas é de aproximadamente R$500,00 reais. Tendo isso em mente, voltei minha atenção a outros fatores que pudessem explicar essa taxa tão baixa de adesão, como por exemplo a forma que a empresa conseguiu contactar o cliente. Analisando os gráficos, percebe-se que os clientes que não foram contactados diretamente pela FortBrasil, ou clientes que a empresa não obteve êxito durante suas tentativas de contato, tiveram um baixíssimo índice de adesão.
 
Estudando os índices de contato, apenas, aproximadamente, 25% dos clientes foram contatados de forma direta, já os outros 75% ou foram contatados de forma indireta, ou a empresa não obteve êxito, ou não foram acionados. Dessa forma, proponho a FortBrasil estudar novos métodos de fazer suas cobranças, como por exemplo, procurar seus clientes em outros veículos de comunicação (whatsapp, telegram e etc), visto que a adesão de seus clientes está diretamente ligada a forma como ele foi contatado.